# Tabnet Image Model

In [2]:
import tabnet
from load_data import *
from helpers_image_pricecat import *
import os
from tqdm import tqdm
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras.constraints import maxnorm
from sklearn.model_selection import train_test_split
import pandas as pd
import matplotlib.pyplot as plt
tf.compat.v1.enable_eager_execution()


# Display
from IPython.display import Image, display
import matplotlib.pyplot as plt
import matplotlib.cm as cm

Init Plugin
Init Graph Optimizer
Init Kernel


/opt/homebrew/Caskroom/miniforge/base/envs/tensorflow_m1/lib/python3.9/site-packages/statsmodels/compat/pandas.py:65: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import Int64Index as NumericIndex


In [122]:
# Load data
X_train, X_test, y_train, y_test = load_data(drop_id=False, skip = False)

------------------------------
Selecting Features data...
------------------------------
------------------------------
Loading data...
------------------------------


/Users/dmnk/OneDrive - stud.uni-goettingen.de/Dokumente/3. Semester/SeminarDL/DubAir/load_data.py:73: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  price = price.str.replace("$","")


Data loaded.


/Users/dmnk/OneDrive - stud.uni-goettingen.de/Dokumente/3. Semester/SeminarDL/DubAir/load_data.py:112: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  listings["host_location_country"][fil] = str(i)
/Users/dmnk/OneDrive - stud.uni-goettingen.de/Dokumente/3. Semester/SeminarDL/DubAir/load_data.py:112: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  listings["host_location_country"][fil] = str(i)
/Users/dmnk/OneDrive - stud.uni-goettingen.de/Dokumente/3. Semester/SeminarDL/DubAir/load_data.py:112: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation:

Data cleansed.
Imputation done. No NaN's are left in the data.
Further Modifications are done.
Text and Open Street Data generated.
Image data loaded.
Due to insignificant t-tests we dropped:
['Bathtub', 'Bed linens', 'Breakfast', 'Cleaning before checkout', 'Dishwasher', 'Elevator', 'Hair dryer', 'Indoor fireplace', 'Private entrance', 'Security cameras on property', 'Single level home', 'Stoves_available', 'Refridgerator_available', 'Body_soap_available', 'Garden_backyard_available', 'Children_Entertainment', 'Workspace', 'Gym_available', 'Coffee_machine_available', 'Dryer_available', 'Washer_available', 'Hot_tub_available', 'Pool_available', 'Wifi_available', 'AC_available', 'heating_available', 'Kitchen_available', 'Safe_available', 'Water_location', 'sound_system_available', 'TV_available', 'Outdoor_stuff', 'Game_consoles', 'Baby_friendly', 'Special_stuff', 'neighbourhood_cleansed_Dn Laoghaire-Rathdown', 'property_type_Entire guesthouse', 'property_type_Entire loft', 'property_typ

In [123]:
# load image data
bag_features = np.load("data/bag_features.npy")
price_cat = np.load("data/price_cat.npy")
ids = np.load("data/ids.npy")

In [124]:
# filter none informative features
f = (bag_features.sum(axis = 0)<1e-7)
bag_features = bag_features[:,~f]

In [125]:
# setup model
class Tabnet_img(tf.keras.Model):

  def __init__(self,feature_columns,
                 num_features=None,
                 feature_dim=32,
                 output_dim=32,
                 num_decision_steps=3,
                 relaxation_factor=1.5,
                 sparsity_coefficient=1e-5,
                 norm_type='group',
                 batch_momentum=0.98,
                 virtual_batch_size=None,
                 num_groups=1,
                 epsilon=1e-5):
    super().__init__()
    
    # image input
    self.resnet_pre = keras.applications.resnet50.preprocess_input
    self.resnet = tf.keras.applications.resnet.ResNet50(include_top=False, weights='imagenet', pooling="avg", input_shape = (None,None,3))
    self.resnet.trainable = False
    
    self.bn1 = tf.keras.layers.BatchNormalization()
    self.drop = tf.keras.layers.Dropout(0)
    self.dense1 = tf.keras.layers.Dense(512, activation=tf.nn.relu, kernel_regularizer = keras.regularizers.l2(0))
    self.dense2 = tf.keras.layers.Dense(32, activation=tf.nn.relu)
    
    self.tabnet = tabnet.TabNet(feature_columns=feature_columns,
                             num_features=num_features,
                             feature_dim=feature_dim,
                             output_dim=output_dim,
                             num_decision_steps=num_decision_steps,
                             relaxation_factor=relaxation_factor,
                             sparsity_coefficient=sparsity_coefficient,
                             norm_type=norm_type,
                             batch_momentum=batch_momentum,
                             virtual_batch_size=virtual_batch_size,
                             num_groups=num_groups,
                             epsilon=epsilon)
   
    self.dense_out = tf.keras.layers.Dense(1)
  def call(self, inputs, training = None):
    # images
    x = self.resnet_pre(inputs[0])
    x = self.resnet(x)
    x = self.bn1(inputs[0])
    x = self.drop(x)
    x = self.dense1(x)
    out1 = self.dense2(x)
    
    # tabnet
    self.activations = self.tabnet(inputs[1], training=training)
    
    # join
    out = tf.keras.layers.concatenate([out1,self.activations])
    return self.dense_out(out)

In [126]:
# store id and features in dataframe
features = [bag_features[x] for x in range(bag_features.shape[0])]
bag_df = pd.DataFrame({"id": ids, "features": features})

In [127]:
# rename columns (for now)
X_train.columns = X_train.columns.str.replace(" ","_")
X_test.columns = X_test.columns.str.replace(" ","_")

In [128]:
X_train["price"] = y_train.values
X_test["price"] = y_test.values

In [129]:
# merge data for tabnet and images to get consistent ids
data_df = pd.merge(bag_df, X_train, on="id", how="inner")
data_df_test = pd.merge(bag_df, X_test, on="id", how="inner")

# extract consistent columns
X_train_new = data_df.filter(col_names)
X_test_new = data_df_test.filter(col_names)

# extract consistent price
y_train_new = data_df["price"]
y_test_new = data_df_test["price"]

In [130]:
# # drop columns that are no longer necessary
# X_train_new.drop("id", axis =1, inplace = True)
# X_test_new.drop("id", axis =1, inplace = True)


In [131]:

# set columns for tabnet
bin_col = [col for col in X_train_new if np.isin(X_train_new[col].unique(), [0, 1]).all()]
num_col = [col for col in X_train_new if ~np.isin(X_train_new[col].unique(), [0, 1]).all()]
col_names = bin_col + num_col
feature_columns = []

for col in col_names:
    feature_columns.append(tf.feature_column.numeric_column(col))

In [132]:
def transform(ds):
    features = tf.unstack(ds["features"])
    prices = ds["price"]
    
    img = ds["images"]
    x = dict(zip(col_names, features))
    y = prices
    return (img, x), y

In [133]:
# make tf dataset
data_train = tf.data.Dataset.from_tensor_slices({"images": np.stack(data_df["features"]),"features": X_train_new, "price": y_train_new})
data_train = data_train.shuffle(6000, seed = 13)
train_dataset = data_train.take(len(data_df))
train_dataset = train_dataset.map(transform)
train_dataset = train_dataset.batch(60)

data_test= tf.data.Dataset.from_tensor_slices({"images": np.stack(data_df_test["features"]),"features": X_test_new, "price": y_test_new})
test_dataset = data_test.take(len(data_df_test))
test_dataset = test_dataset.map(transform)
test_dataset = test_dataset.batch(60)

In [134]:
len(col_names)

42

In [135]:
model = Tabnet_img(feature_columns = feature_columns, feature_dim = 42, output_dim =32)

[TabNet]: 10 features will be used for decision steps.


In [136]:
def R_squared(y, y_pred):
  residual = tf.reduce_sum(tf.square(tf.subtract(y, y_pred)))
  total = tf.reduce_sum(tf.square(tf.subtract(y, tf.reduce_mean(y))))
  r2 = tf.subtract(1.0, tf.math.divide(residual, total))
  
  return r2

In [137]:
lr = tf.keras.optimizers.schedules.ExponentialDecay(0.01, decay_steps=100, decay_rate=0.9, staircase=False)
optimizer = tf.keras.optimizers.Adam(lr)
model.compile(optimizer, loss="mse", metrics=R_squared)


model.fit(train_dataset, validation_data = test_dataset, epochs=100, verbose=1)

Epoch 1/100


2022-02-02 13:04:46.158074: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


83/83 [==============================] - ETA: 0s - loss: 3.9533 - R_squared: -6.4239

2022-02-02 13:04:53.829804: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


83/83 [==============================] - 10s 93ms/step - loss: 3.9533 - R_squared: -6.4239 - val_loss: 1.0087 - val_R_squared: -0.9259
Epoch 2/100
83/83 [==============================] - 7s 86ms/step - loss: 0.4514 - R_squared: 0.2395 - val_loss: 0.5291 - val_R_squared: 0.0300
Epoch 3/100
83/83 [==============================] - 7s 86ms/step - loss: 0.4083 - R_squared: 0.3112 - val_loss: 0.4312 - val_R_squared: 0.2173
Epoch 4/100
83/83 [==============================] - 7s 86ms/step - loss: 0.3830 - R_squared: 0.3599 - val_loss: 0.3467 - val_R_squared: 0.3688
Epoch 5/100
83/83 [==============================] - 7s 86ms/step - loss: 0.3592 - R_squared: 0.3978 - val_loss: 0.3668 - val_R_squared: 0.3346
Epoch 6/100
83/83 [==============================] - 7s 86ms/step - loss: 0.3326 - R_squared: 0.4340 - val_loss: 0.3643 - val_R_squared: 0.3371
Epoch 7/100
83/83 [==============================] - 7s 87ms/step - loss: 0.3094 - R_squared: 0.4867 - val_loss: 0.3168 - val_R_squared: 0.4097
E

# Try with collage data

In [57]:
# Load data
X_train, X_test, y_train, y_test = load_data(drop_id=False, skip = False)

------------------------------
Selecting Features data...
------------------------------
------------------------------
Loading data...
------------------------------


/Users/dmnk/OneDrive - stud.uni-goettingen.de/Dokumente/3. Semester/SeminarDL/DubAir/load_data.py:73: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  price = price.str.replace("$","")


Data loaded.


/Users/dmnk/OneDrive - stud.uni-goettingen.de/Dokumente/3. Semester/SeminarDL/DubAir/load_data.py:112: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  listings["host_location_country"][fil] = str(i)
/Users/dmnk/OneDrive - stud.uni-goettingen.de/Dokumente/3. Semester/SeminarDL/DubAir/load_data.py:112: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  listings["host_location_country"][fil] = str(i)
/Users/dmnk/OneDrive - stud.uni-goettingen.de/Dokumente/3. Semester/SeminarDL/DubAir/load_data.py:112: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation:

Data cleansed.
Imputation done. No NaN's are left in the data.
Further Modifications are done.
Text and Open Street Data generated.
Image data loaded.
Due to insignificant t-tests we dropped:
['Bathtub', 'Bed linens', 'Breakfast', 'Cleaning before checkout', 'Dishwasher', 'Elevator', 'Hair dryer', 'Indoor fireplace', 'Private entrance', 'Security cameras on property', 'Single level home', 'Stoves_available', 'Refridgerator_available', 'Body_soap_available', 'Garden_backyard_available', 'Children_Entertainment', 'Workspace', 'Gym_available', 'Coffee_machine_available', 'Dryer_available', 'Washer_available', 'Hot_tub_available', 'Pool_available', 'Wifi_available', 'AC_available', 'heating_available', 'Kitchen_available', 'Safe_available', 'Water_location', 'sound_system_available', 'TV_available', 'Outdoor_stuff', 'Game_consoles', 'Baby_friendly', 'Special_stuff', 'neighbourhood_cleansed_Dn Laoghaire-Rathdown', 'property_type_Entire guesthouse', 'property_type_Entire loft', 'property_typ

In [58]:

images = np.load("data/images_collage.npy", allow_pickle = True)
ids = np.load("data/ids_collage.npy").astype(int)
price = np.load("data/price_collage.npy", allow_pickle = True)

In [59]:
# setup model
class Tabnet_img(tf.keras.Model):

  def __init__(self,feature_columns,
                 num_features=None,
                 feature_dim=32,
                 output_dim=32,
                 num_decision_steps=3,
                 relaxation_factor=1.5,
                 sparsity_coefficient=1e-5,
                 norm_type='group',
                 batch_momentum=0.98,
                 virtual_batch_size=None,
                 num_groups=1,
                 epsilon=1e-5):
    super().__init__()
    
    # image input
    self.resnet_pre = keras.applications.resnet50.preprocess_input
    self.resnet = tf.keras.applications.resnet.ResNet50(include_top=False, weights='imagenet', pooling="avg", input_shape = (None,None,3))
    self.resnet.trainable = False
    
    self.bn1 = tf.keras.layers.BatchNormalization()
    self.drop = tf.keras.layers.Dropout(0)
    self.dense1 = tf.keras.layers.Dense(512, activation=tf.nn.relu, kernel_regularizer = keras.regularizers.l2(0))
    self.dense2 = tf.keras.layers.Dense(32, activation=tf.nn.relu)
    
    self.tabnet = tabnet.TabNet(feature_columns=feature_columns,
                             num_features=num_features,
                             feature_dim=feature_dim,
                             output_dim=output_dim,
                             num_decision_steps=num_decision_steps,
                             relaxation_factor=relaxation_factor,
                             sparsity_coefficient=sparsity_coefficient,
                             norm_type=norm_type,
                             batch_momentum=batch_momentum,
                             virtual_batch_size=virtual_batch_size,
                             num_groups=num_groups,
                             epsilon=epsilon)
   
    self.dense_out = tf.keras.layers.Dense(1)
  def call(self, inputs, training = None, train_resnet = False):
    if train_resnet:
      self.resnet.trainable = True
    # images
    x = tf.cast(inputs[0], dtype=tf.float32)
    x = self.resnet_pre(inputs[0])
    x = self.resnet(x)
    x = self.bn1(inputs[0])
    x = self.drop(x)
    x = self.dense1(x)
    out1 = self.dense2(x)
    
    # tabnet
    self.activations = self.tabnet(inputs[1], training=training)
    
    # join
    out = tf.keras.layers.concatenate([out1,self.activations])
    return self.dense_out(out)

In [60]:
# store id and images in dataframe
features = [images[x] for x in range(images.shape[0])]
images_df = pd.DataFrame({"id": ids, "images": features})

In [61]:
# rename columns (for now)
X_train.columns = X_train.columns.str.replace(" ","_")
X_test.columns = X_test.columns.str.replace(" ","_")

In [44]:

# set columns for tabnet
bin_col = [col for col in X_train if np.isin(X_train[col].unique(), [0, 1]).all()]
num_col = [col for col in X_train if ~np.isin(X_train[col].unique(), [0, 1]).all()]
col_names = bin_col + num_col
col_names.remove("id")
feature_columns = []

for col in col_names:
    feature_columns.append(tf.feature_column.numeric_column(col))

In [45]:
X_train["price"] = y_train.values
X_test["price"] = y_test.values

In [46]:
# merge data for tabnet and images to get consistent ids
data_df = pd.merge(images_df, X_train, on="id", how="inner")
data_df_test = pd.merge(images_df, X_test, on="id", how="inner")

# extract consistent columns
X_train_new = data_df.filter(col_names)
X_test_new = data_df_test.filter(col_names)

# extract consistent price
y_train_new = data_df["price"]
y_test_new = data_df_test["price"]

In [56]:
# # drop columns that are no longer necessary
X_train_new.drop("id", axis =1, inplace = True)
X_test_new.drop("id", axis =1, inplace = True)


KeyError: "['id'] not found in axis"

In [48]:
def transform(ds):
    features = tf.unstack(ds["features"])
    prices = ds["price"]
    
    img = ds["images"]
    x = dict(zip(col_names, features))
    y = prices
    return (img, x), y

In [49]:
# make tf dataset
data_train = tf.data.Dataset.from_tensor_slices({"images": np.stack(data_df["images"]),"features": X_train_new, "price": y_train_new})
data_train = data_train.shuffle(6000, seed = 13)
train_dataset = data_train.take(len(data_df))
train_dataset = train_dataset.map(transform)
train_dataset = train_dataset.batch(60)

data_test= tf.data.Dataset.from_tensor_slices({"images": np.stack(data_df_test["images"]),"features": X_test_new, "price": y_test_new})
test_dataset = data_test.take(len(data_df_test))
test_dataset = test_dataset.map(transform)
test_dataset = test_dataset.batch(60)

In [55]:
len(col_names)

42

In [52]:
model = Tabnet_img(feature_columns = feature_columns, feature_dim = 42, output_dim =32)

[TabNet]: 10 features will be used for decision steps.


In [53]:
def R_squared(y, y_pred):
  residual = tf.reduce_sum(tf.square(tf.subtract(y, y_pred)))
  total = tf.reduce_sum(tf.square(tf.subtract(y, tf.reduce_mean(y))))
  r2 = tf.subtract(1.0, tf.math.divide(residual, total))
  
  return r2

In [54]:
lr = tf.keras.optimizers.schedules.ExponentialDecay(0.01, decay_steps=100, decay_rate=0.9, staircase=False)
optimizer = tf.keras.optimizers.Adam(lr)
model.compile(optimizer, loss="mse", metrics=R_squared)


model.fit(train_dataset, validation_data = test_dataset, epochs=100, verbose=1)

Epoch 1/100


TypeError: in user code:

    /opt/homebrew/Caskroom/miniforge/base/envs/tensorflow_m1/lib/python3.9/site-packages/tensorflow/python/keras/engine/training.py:855 train_function  *
        return step_function(self, iterator)
    /var/folders/97/j215pw6x7sq158bvx1ktlhf80000gn/T/ipykernel_1490/3916806783.py:49 call  *
        x = self.bn1(inputs[0])
    /opt/homebrew/Caskroom/miniforge/base/envs/tensorflow_m1/lib/python3.9/site-packages/tensorflow/python/keras/engine/base_layer.py:1030 __call__  **
        outputs = call_fn(inputs, *args, **kwargs)
    /opt/homebrew/Caskroom/miniforge/base/envs/tensorflow_m1/lib/python3.9/site-packages/tensorflow/python/keras/layers/normalization.py:772 call
        outputs = self._fused_batch_norm(inputs, training=training)
    /opt/homebrew/Caskroom/miniforge/base/envs/tensorflow_m1/lib/python3.9/site-packages/tensorflow/python/keras/layers/normalization.py:625 _fused_batch_norm
        output, mean, variance = control_flow_util.smart_cond(
    /opt/ho